In [1]:
from tensorflow import keras, expand_dims
import numpy as np

In [2]:
VisFin = keras.Sequential()
VisFin.add(keras.layers.Conv2D(9, kernel_size=(9, 9),padding='same', input_shape=(201, 43, 1), activation='relu'))
VisFin.add(keras.layers.Conv2D(9, kernel_size = (9,9),padding='same', activation='relu'))
VisFin.add(keras.layers.MaxPooling2D())
VisFin.add(keras.layers.Conv2D(9, kernel_size = (6,6),padding='same', activation='relu'))
VisFin.add(keras.layers.MaxPooling2D())
VisFin.add(keras.layers.Flatten())
VisFin.add(keras.layers.Dense(42, activation='relu'))
VisFin.add(keras.layers.Dense(7,activation='softmax'))

In [3]:
VisFin.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
# categorical_crossentropy

In [4]:
from modules.data_feed import *
modeling = prepare_data('historic_data.db', 'returns')

Connected to returns table!


In [5]:
for tick in range(5):
    data = modeling.mass_hacking(tick=tick)
    tr_data = expand_dims(np.array(data[0]),axis=-1)
    tr_key = np.array(data[1])
    VisFin.fit(tr_data, tr_key,epochs=10, verbose=2)
VisFin.save('VisFin_v0.h5')

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
